In [7]:
import pandas as pd
import sqlite3

# Load CSV

orders = pd.read_csv(r"C:\Users\Janu V\Documents\Hackthon_Soln\data\orders.csv")

print("Orders shape:", orders.shape)
orders.head()



Orders shape: (10000, 6)


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [8]:
users = pd.read_json("C:/Users/Janu V/Documents/Hackthon_Soln/data/users.json")
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [9]:
import sqlite3

conn = sqlite3.connect(":memory:")

# Load SQL

with open("C:/Users/Janu V/Documents/Hackthon_Soln/data/restaurants.sql", "r") as f:
    sql_script = f.read()

conn.executescript(sql_script)

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [10]:
merged = orders.merge(users, on="user_id", how="left")

final_df = merged.merge(
    restaurants,
    on="restaurant_id",
    how="left"
)

final_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [11]:
final_df.to_csv(
    "C:/Users/Janu V/Documents/Hackthon_Soln/output/final_food_delivery_dataset.csv",
    index=False
)


In [12]:
# Which city has the highest total revenue (total_amount) from Gold members?
final_df.query("membership=='Gold'").groupby("city")["total_amount"].sum().idxmax()


'Chennai'

In [17]:
# Which cuisine has the highest average order value across all orders?

final_df.groupby("cuisine")["total_amount"].mean().idxmax()



'Mexican'

In [19]:
# How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)?
(final_df.groupby("user_id")["total_amount"].sum() > 1000).sum()



np.int64(2544)

In [20]:
# Which restaurant rating range generated the highest total revenue?
rating_ranges = pd.cut(final_df["rating"], bins=8)

answer = (
    final_df
    .groupby(rating_ranges)["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

answer.head(1)


rating
(4.75, 5.0]    1284861.27
Name: total_amount, dtype: float64

In [21]:
# Among Gold members, which city has the highest average order value?
gold_city_answer = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

gold_city_answer.head(1)


city
Chennai    808.45908
Name: total_amount, dtype: float64

In [22]:
# Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?
final_df.groupby("cuisine").agg(restaurants=("restaurant_id","nunique"), revenue=("total_amount","sum")).query("revenue >= revenue.median()").sort_values(["restaurants","revenue"], ascending=[True, False]).head(1)


,restaurants,revenue
cuisine,,
Italian,126,2024203.8


In [23]:
# What percentage of total orders were placed by Gold members? (Rounded to nearest integer)
round((final_df["membership"].eq("Gold").sum() / len(final_df)) * 100)



50

In [39]:
# Which restaurant has the highest average order value but less than 20 total orders (from given options)?
options = ["Grand Cafe Punjabi","Grand Restaurant South Indian","Ruchi Mess Multicuisine","Ruchi Foods Chinese"]
name_col = [c for c in final_df.columns if "name" in c.lower() and "rest" in c.lower()][0]
s = final_df[final_df[name_col].isin(options)].groupby(name_col)["total_amount"].agg(["mean","count"])
s[s["count"] < 20].sort_values("mean", ascending=False).index[0]





'Ruchi Foods Chinese'

In [43]:
# Which combination contributes the highest revenue? (restricted to given options)
opts = [("Gold","Indian"),("Gold","Italian"),("Regular","Indian"),("Regular","Chinese")]
final_df.groupby(["membership","cuisine"])["total_amount"].sum().loc[opts].sort_values(ascending=False).index[0]





('Gold', 'Italian')

In [41]:
# During which quarter of the year is the total revenue highest?
"Q" + str(final_df.assign(q=pd.to_datetime(final_df["order_date"], dayfirst=True).dt.quarter).groupby("q")["total_amount"].sum().idxmax())



'Q3'

In [30]:
# How many total orders were placed by users with Gold membership?
final_df["membership"].eq("Gold").sum()


np.int64(4987)

In [31]:
# What is the total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city?
round(final_df.loc[final_df["city"]=="Hyderabad","total_amount"].sum())


1889367

In [32]:
# How many distinct users placed at least one order?
final_df["user_id"].nunique()


2883

In [33]:
# What is the average order value (rounded to 2 decimals) for Gold members?
round(final_df.loc[final_df["membership"]=="Gold","total_amount"].mean(), 2)


np.float64(797.15)

In [34]:
# How many orders were placed for restaurants with rating ≥ 4.5?
final_df.loc[final_df["rating"]>=4.5].shape[0]


3374

In [35]:
# How many orders were placed in the top revenue city among Gold members only?
g = final_df[final_df["membership"]=="Gold"]
g[g["city"]==g.groupby("city")["total_amount"].sum().idxmax()].shape[0]


1337